In [1]:
import pandas as pd
import numpy as np

## Get ids, labels, some road features

In [2]:
accident_df = pd.read_csv("accidents-route.csv")

c:\Users\rahar\.conda\envs\mlenv\lib\site-packages\IPython\core\interactiveshell.py:3441: DtypeWarning: Columns (12,15) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
accident_df.columns

Index(['code-infraction', 'code-postal', 'date-creation-evn', 'date-fermeture',
       'date-debut-infraction', 'annee-infraction', 'mois-infraction',
       'date-infraction', 'date-fin-infraction', 'direction',
       'montant-dommage', 'numero-evenement', 'equipe', 'heure-debut',
       'heure-fin', 'secteur-ilot', 'inscription-mip', 'code-municipal',
       'nombre-personnes-dossier', 'nombre-vehicules-dossier',
       'numero-civique', 'province', 'type-rue', 'rue', 'rue-2', 'type-rue-2',
       'annee-enrg-dossier', 'mois-enrg-dossier', 'date-enrg-dossier', 'pdq',
       'vehicule-annee', 'vehicule-couleur-1', 'vehicule-genre', 'province-1',
       'vehicule-style', 'vehicule-compagnie-assurance', 'vehicule-marque',
       'vehicule-modele', 'vehicule-description'],
      dtype='object')

In [4]:
accident_df.head()

,code-infraction,code-postal,date-creation-evn,date-fermeture,date-debut-infraction,annee-infraction,mois-infraction,date-infraction,date-fin-infraction,direction,...,pdq,vehicule-annee,vehicule-couleur-1,vehicule-genre,province-1,vehicule-style,vehicule-compagnie-assurance,vehicule-marque,vehicule-modele,vehicule-description
0,Délit de fuite CSR,H7V3A2,2019-07-01,2019-07-02,2019-06-30,2019,6,30,2019-07-01,,...,PDQ3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Collision matérielle de 2 000 $ et -,NaN,2019-06-30,2019-07-01,2019-06-30,2019,6,30,NaN,,...,PDQ3,2010.0,NaN,VUS,QUE,NaN,AVIVA,HYUN,SANT,Moyen
2,Collision matérielle de 2 000 $ et -,NaN,2019-06-30,2019-07-01,2019-06-30,2019,6,30,NaN,,...,PDQ3,2013.0,NaN,CAMIO,QUE,NaN,NaN,EONE,NaN,Grand
3,Collision matérielle de + de 2 000 $,NaN,2019-06-30,2019-07-05,2019-06-30,2019,6,30,NaN,,...,PDQ3,2007.0,ROU,AUTO,QUE,FGNET,INTACT,DODG,CARA,Moyen
4,Collision matérielle de + de 2 000 $,NaN,2019-06-30,2019-07-05,2019-06-30,2019,6,30,NaN,,...,PDQ3,2018.0,BLA,CAMIO,QUE,FERMEE,INTACT,GMC,SAVA,Moyen


In [5]:
infraction_types = accident_df["code-infraction"].unique()
infraction_types

array(['Délit de fuite CSR', 'Collision matérielle de 2 000 $ et -',
       'Collision matérielle de + de 2 000 $', 'Collision avec blessé',
       'Collision mortelle', 'Collision hors route de + de 2 000 $',
       'Collision hors route avec blessé',
       'Collision hors route de 2 000 $ et -',
       'Collision mortelle hors route',
       'Incendie de véhicule en mouvement'], dtype=object)

In [6]:
collisions = ['Collision matérielle de 2 000 $ et -',
       'Collision matérielle de + de 2 000 $', 'Collision avec blessé',
       'Collision mortelle', 'Collision hors route de + de 2 000 $',
       'Collision hors route avec blessé',
       'Collision hors route de 2 000 $ et -',
       'Collision mortelle hors route']

collisions_df = accident_df[accident_df["code-infraction"].isin(collisions)]

In [7]:
print(collisions_df.shape)
collisions_df.head(3)

(24230, 39)


,code-infraction,code-postal,date-creation-evn,date-fermeture,date-debut-infraction,annee-infraction,mois-infraction,date-infraction,date-fin-infraction,direction,...,pdq,vehicule-annee,vehicule-couleur-1,vehicule-genre,province-1,vehicule-style,vehicule-compagnie-assurance,vehicule-marque,vehicule-modele,vehicule-description
1,Collision matérielle de 2 000 $ et -,NaN,2019-06-30,2019-07-01,2019-06-30,2019,6,30,NaN,,...,PDQ3,2010.0,NaN,VUS,QUE,NaN,AVIVA,HYUN,SANT,Moyen
2,Collision matérielle de 2 000 $ et -,NaN,2019-06-30,2019-07-01,2019-06-30,2019,6,30,NaN,,...,PDQ3,2013.0,NaN,CAMIO,QUE,NaN,NaN,EONE,NaN,Grand
3,Collision matérielle de + de 2 000 $,NaN,2019-06-30,2019-07-05,2019-06-30,2019,6,30,NaN,,...,PDQ3,2007.0,ROU,AUTO,QUE,FGNET,INTACT,DODG,CARA,Moyen


In [8]:
collisions_2018 = collisions_df[collisions_df["annee-infraction"] == 2018]
print(collisions_2018.shape)
collisions_2018.head(3)

(5312, 39)


,code-infraction,code-postal,date-creation-evn,date-fermeture,date-debut-infraction,annee-infraction,mois-infraction,date-infraction,date-fin-infraction,direction,...,pdq,vehicule-annee,vehicule-couleur-1,vehicule-genre,province-1,vehicule-style,vehicule-compagnie-assurance,vehicule-marque,vehicule-modele,vehicule-description
375,Collision avec blessé,NaN,2019-06-06,2019-06-11,2018-12-13,2018,12,13,NaN,,...,PDQ3,NaN,BLA,AUTO,QUE,NaN,NaN,NaN,NaN,Petit
3132,Collision matérielle de + de 2 000 $,NaN,2019-01-17,2019-01-20,2018-01-17,2018,1,17,NaN,,...,PDQ5,2018.0,NaN,VUS,QUE,CMNET,DESJARDINS,NISS,ROGU,Petit
3179,Collision avec blessé,NaN,2019-01-14,2019-01-17,2018-01-14,2018,1,14,NaN,,...,PDQ3,2013.0,NOI,AUTO,QUE,4PORT,Wawanesa,NISS,MURA,Moyen


In [9]:
collisions_2018.columns

Index(['code-infraction', 'code-postal', 'date-creation-evn', 'date-fermeture',
       'date-debut-infraction', 'annee-infraction', 'mois-infraction',
       'date-infraction', 'date-fin-infraction', 'direction',
       'montant-dommage', 'numero-evenement', 'equipe', 'heure-debut',
       'heure-fin', 'secteur-ilot', 'inscription-mip', 'code-municipal',
       'nombre-personnes-dossier', 'nombre-vehicules-dossier',
       'numero-civique', 'province', 'type-rue', 'rue', 'rue-2', 'type-rue-2',
       'annee-enrg-dossier', 'mois-enrg-dossier', 'date-enrg-dossier', 'pdq',
       'vehicule-annee', 'vehicule-couleur-1', 'vehicule-genre', 'province-1',
       'vehicule-style', 'vehicule-compagnie-assurance', 'vehicule-marque',
       'vehicule-modele', 'vehicule-description'],
      dtype='object')

In [10]:
from pyexpat import features


features = ["code-infraction","mois-infraction","type-rue","rue","type-rue-2"]
collisions_2018 = collisions_2018[features].dropna()

In [11]:
collisions_2018.head()

,code-infraction,mois-infraction,type-rue,rue,type-rue-2
375,Collision avec blessé,12,BOULEV,CHOMEDEY,BOULEV
3132,Collision matérielle de + de 2 000 $,1,BOULEV,CURE-LABELLE,BOULEV
3179,Collision avec blessé,1,RUE,CLARENDON,BOULEV
3180,Collision avec blessé,1,RUE,CLARENDON,BOULEV
3312,Collision matérielle de 2 000 $ et -,12,VOIE,LAVAL EST A-440,AUTORO


In [12]:
type_rue = collisions_2018["type-rue-2"].unique()
rues = collisions_2018["rue"].unique()
print(type_rue)
print(len(rues))
print(collisions_2018.shape[0]/len(rues)/12)

['BOULEV' 'AUTORO' 'AVENUE' 'RUE' 'PLACE' 'MONTEE' 'CHEMIN' 'VOIE'
 'PROMEN' 'CROISS' 'ROUTE' 'TERRAS' 'RANG' 'VIADUC' 'ECHANG']
267
0.8898252184769039


The average for accident per road per month is almost one so month seems like a good fit for time period.

In [13]:
collisions_2018["id"] = collisions_2018["rue"] + "@" + collisions_2018["mois-infraction"].astype(str)

In [14]:
len(collisions_2018["id"].unique())

698

In [15]:
uniques = collisions_2018["id"].unique()

In [16]:
# count = collisions_2018["id"].value_counts()
_count = collisions_2018.groupby(["id"])["id"].count()
print(_count.shape)
_count.head(3)

(698,)


id
100E@10    4
100E@11    6
100E@12    4
Name: id, dtype: int64

In [17]:
months = [int(i.split("@")[1]) for i in uniques]
road = [i.split("@")[0] for i in uniques]
road_type = [collisions_2018[collisions_2018["id"] == i]["type-rue"].iloc[0] for i in uniques]

In [18]:
count = pd.DataFrame({"id":uniques,"count":_count[uniques],"month":months,"road":road,"road_type":road_type})
count.reset_index(drop=True, inplace=True)
count.head()

,id,count,month,road,road_type
0,CHOMEDEY@12,10,12,CHOMEDEY,BOULEV
1,CURE-LABELLE@1,23,1,CURE-LABELLE,BOULEV
2,CLARENDON@1,2,1,CLARENDON,RUE
3,LAVAL EST A-440@12,10,12,LAVAL EST A-440,VOIE
4,MARCEL-VILLENEUVE@12,2,12,MARCEL-VILLENEUVE,AVENUE


In [19]:
count.shape

(698, 5)

## Getting speed limits in the df

In [20]:
speed_raw = pd.read_csv("limite-vitesse-2014.csv")
print(speed_raw.shape)
speed_raw.head(5)

(15464, 11)


,WKT,ID_TRC,T_VOIE,L_VOIE,N_VOIE,D_VOIE,LI_GCH,LI_DRT,ODONYME,ODONYME_CO,LIMITE_VIT
0,LINESTRING (-73.6798629184971 45.5559610468134...,508381,Boulevard,des,Prairies,NaN,Laval-des-Rapides,Laval-des-Rapides,Boulevard des Prairies,Boul. des Prairies,50
1,LINESTRING (-73.6806608913008 45.5555611608213...,508380,Boulevard,des,Prairies,NaN,Laval-des-Rapides,Laval-des-Rapides,Boulevard des Prairies,Boul. des Prairies,50
2,LINESTRING (-73.6814206363064 45.5552261180531...,508379,Boulevard,des,Prairies,NaN,Laval-des-Rapides,Laval-des-Rapides,Boulevard des Prairies,Boul. des Prairies,50
3,LINESTRING (-73.6822380588474 45.5549062754073...,508378,Boulevard,des,Prairies,NaN,Laval-des-Rapides,Laval-des-Rapides,Boulevard des Prairies,Boul. des Prairies,50
4,LINESTRING (-73.6830030082074 45.5546018081614...,508377,Boulevard,des,Prairies,NaN,Laval-des-Rapides,Laval-des-Rapides,Boulevard des Prairies,Boul. des Prairies,50


In [21]:
odonymes = speed_raw["ODONYME"].unique()
print(len(odonymes))
print(odonymes[0:5])

3023
['Boulevard des Prairies' 'Boulevard Cléroux' "Rang de l'Équerre"
 'Montée Major' "Rue d'Aurillac"]


In [22]:
lim_speed_by_od = {}
for index,row in speed_raw.iterrows():
    if row[8] not in lim_speed_by_od.keys():
        lim_speed_by_od[row[8]] = row[-1]

In [23]:
set(lim_speed_by_od.keys()) == set(odonymes)

True

In [24]:
check = []
welp = []
matches_dict = {}
for i in rues:
    for j in odonymes:
        if i.lower() in j.lower():
            check.append(True)
            welp.append(True)
            matches_dict[i] = lim_speed_by_od[j]
            break
    check.append(False)

In [25]:
count["speed_limit"] = count["road"].map(matches_dict)
count["speed_limit"] = count["speed_limit"].fillna(0)

In [26]:
count.head()

,id,count,month,road,road_type,speed_limit
0,CHOMEDEY@12,10,12,CHOMEDEY,BOULEV,0.0
1,CURE-LABELLE@1,23,1,CURE-LABELLE,BOULEV,0.0
2,CLARENDON@1,2,1,CLARENDON,RUE,40.0
3,LAVAL EST A-440@12,10,12,LAVAL EST A-440,VOIE,0.0
4,MARCEL-VILLENEUVE@12,2,12,MARCEL-VILLENEUVE,AVENUE,70.0


The second approach seems to offer a good correspondance (91.4405%)
Ok now it'S 110% so there are double matches

In [27]:
count = count.dropna()

In [28]:
count

,id,count,month,road,road_type,speed_limit
0,CHOMEDEY@12,10,12,CHOMEDEY,BOULEV,0.0
1,CURE-LABELLE@1,23,1,CURE-LABELLE,BOULEV,0.0
2,CLARENDON@1,2,1,CLARENDON,RUE,40.0
3,LAVAL EST A-440@12,10,12,LAVAL EST A-440,VOIE,0.0
4,MARCEL-VILLENEUVE@12,2,12,MARCEL-VILLENEUVE,AVENUE,70.0
...,...,...,...,...,...,...
693,ROMAIN@1,2,1,ROMAIN,RUE,40.0
694,2E@1,2,1,2E,AVENUE,40.0
695,DU CALVET@1,2,1,DU CALVET,RUE,40.0
696,SABOURIN@1,2,1,SABOURIN,RUE,40.0


## Add weather data

In [29]:
weather_raw = pd.read_csv("weather_compilation.csv")
weather_raw.head(5)

,Unnamed: 0,year,month,temp,rain,snow
0,0,2017.0,1.0,-4.547,1.819,8.420
1,1,2017.0,2.0,-3.697,3.188,13.000
2,2,2017.0,3.0,-4.030,2.666,12.260
3,3,2017.0,4.0,7.977,5.094,1.154
4,4,2017.0,5.0,13.310,3.674,0.000


In [30]:
weather_2018 = weather_raw[weather_raw["year"] == 2018]
weather_2018.head()

,Unnamed: 0,year,month,temp,rain,snow
12,12,2018.0,1.0,-9.7200,2.916,20.19
13,13,2018.0,2.0,-4.5620,2.182,21.58
14,14,2018.0,3.0,-0.9194,1.345,6.97
15,15,2018.0,4.0,3.9200,3.120,2.20
16,16,2018.0,5.0,15.8360,1.564,0.00


In [31]:
for i in ["temp","rain","snow"]:
    d = {i:j for i,j in zip(weather_2018["month"],weather_2018[i])}
    count[i] = count["month"].map(d)

In [32]:
count

,id,count,month,road,road_type,speed_limit,temp,rain,snow
0,CHOMEDEY@12,10,12,CHOMEDEY,BOULEV,0.0,-4.977,3.137,1.261
1,CURE-LABELLE@1,23,1,CURE-LABELLE,BOULEV,0.0,-9.720,2.916,20.190
2,CLARENDON@1,2,1,CLARENDON,RUE,40.0,-9.720,2.916,20.190
3,LAVAL EST A-440@12,10,12,LAVAL EST A-440,VOIE,0.0,-4.977,3.137,1.261
4,MARCEL-VILLENEUVE@12,2,12,MARCEL-VILLENEUVE,AVENUE,70.0,-4.977,3.137,1.261
...,...,...,...,...,...,...,...,...,...
693,ROMAIN@1,2,1,ROMAIN,RUE,40.0,-9.720,2.916,20.190
694,2E@1,2,1,2E,AVENUE,40.0,-9.720,2.916,20.190
695,DU CALVET@1,2,1,DU CALVET,RUE,40.0,-9.720,2.916,20.190
696,SABOURIN@1,2,1,SABOURIN,RUE,40.0,-9.720,2.916,20.190
